# Laboratorio 3
Ignacio Mendez (22613) y Diego Soto (22737)

## Task 1

#### Carga de librerias

In [1]:
import numpy as np
import math
import cv2

#### Matrices a usar

In [2]:
M = np.array([
    [120, 5],
    [5, 115]
])

M2 = np.array([
    [200, 10],
    [10, 1]
])


#### Creación de funciones

In [ ]:
def autovalores_2x2(A):

    a = A[0][0]
    b = A[0][1]
    c = A[1][0]
    d = A[1][1]

    tr = a + d
    det = a*d - b*c

    disc = tr**2 - 4*det

    lambda1 = (tr + math.sqrt(disc)) / 2
    lambda2 = (tr - math.sqrt(disc)) / 2

    return lambda1, lambda2


In [4]:
def harris(M, k=0.04):
    m11 = M[0][0]
    m12 = M[0][1]
    m22 = M[1][1]

    detM = m11 * m22 - m12 * m12
    trM = m11 + m22

    R = detM - k * (trM ** 2)
    return R


#### Matriz M

$$
M =
\begin{pmatrix}
120 & 5 \\
5 & 115
\end{pmatrix}
$$

### Eigenvalores

$$
\operatorname{tr}(M) = 120 + 115 = 235
$$

$$
\det(M) = 120 \cdot 115 - 5 \cdot 5 = 13775
$$

$$
\operatorname{tr}(M)^2 - 4\det(M) = 235^2 - 4(13775) = 125
$$

$$
\sqrt{125} \approx 11.1803
$$

$$
\lambda_1 = \frac{235 + 11.1803}{2} \approx 123.0902
$$

$$
\lambda_2 = \frac{235 - 11.1803}{2} \approx 111.9098
$$

### Respuesta de Harris

$$
R = \det(M) - k(\operatorname{tr}(M))^2
$$

$$
R = 13775 - 0.04(235^2)
$$

$$
R = 13775 - 2209 = 11566
$$

In [6]:
eig_M = autovalores_2x2(M)
harris_M = harris(M)

print("Autovalores matriz M:", eig_M)
print("Respuesta de Harris matriz M:", harris_M)

Autovalores matriz M: (123.09016994374947, 111.90983005625053)
Respuesta de Harris matriz M: 11566.0


Para la matriz M se puede ver que ambos autovalores son altos indicando cambios fuertes en ambas direcciones, la respuesta de Harris es positiva y alta. Esto indica que el pixel analizado en la matriz M es una **esquina**. 

#### Matriz M'

$$
M' =
\begin{pmatrix}
200 & 10 \\
10 & 1
\end{pmatrix}
$$

### Eigenvalores

$$
\operatorname{tr}(M') = 200 + 1 = 201
$$

$$
\det(M') = 200 \cdot 1 - 10 \cdot 10 = 100
$$

$$
\operatorname{tr}(M')^2 - 4\det(M') = 201^2 - 4(100) = 40001
$$

$$
\sqrt{40001} \approx 200.0025
$$

$$
\lambda_1 = \frac{201 + 200.0025}{2} \approx 200.50125
$$

$$
\lambda_2 = \frac{201 - 200.0025}{2} \approx 0.49875
$$

### Respuesta de Harris

$$
R = \det(M') - k(\operatorname{tr}(M'))^2
$$

$$
R = 100 - 0.04(201^2)
$$

$$
R = 100 - 1616.04 = -1516.04
$$

In [7]:
eig_M2 = autovalores_2x2(M2)
harris_M2 = harris(M2)

print("Autovalores matriz M': ", eig_M2)
print("Respuesta de Harris matriz M': ", harris_M2)

Autovalores matriz M':  (200.50124999218758, 0.4987500078124043)
Respuesta de Harris matriz M':  -1516.04


Para el caso de la matriz M' se puede ver que un autovalor es grande y el otro pequeño, indicando cambio fuerte en una sola dirección, la respuesta de Harris es negativa. Esto indica que el pixel analizado en la matriz M' es un **borde**. 

## Task 2

In [3]:
imagen1 = cv2.imread("imagen1.png", cv2.IMREAD_GRAYSCALE)
imagen2 = cv2.imread("imagen2.png", cv2.IMREAD_GRAYSCALE)

In [4]:
# Crear el detector SIFT
sift = cv2.SIFT_create()

# Detectar keypoints y descriptores en las dos imágenes
kp1_sift, des1_sift = sift.detectAndCompute(imagen1, None)
kp2_sift, des2_sift = sift.detectAndCompute(imagen2, None)

# BFMatcher con distancia Euclidiana
bf_sift = cv2.BFMatcher(cv2.NORM_L2)

# Buscar los dos mejores matches por descriptor
matches_sift = bf_sift.knnMatch(des1_sift, des2_sift, k=2)

good_sift = []

for m, n in matches_sift:
    # Si el mejor match es mucho mejor que el segundo
    if m.distance < 0.75 * n.distance:
        good_sift.append(m)

img_sift = cv2.drawMatches(
    imagen1, kp1_sift,
    imagen2, kp2_sift,
    good_sift, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

In [13]:
# Crear el detector ORB
orb = cv2.ORB_create(nfeatures=2000)

kp1_orb, des1_orb = orb.detectAndCompute(imagen1, None)
kp2_orb, des2_orb = orb.detectAndCompute(imagen2, None)

# BFMatcher con distancia Hamming
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING)

matches_orb = bf_orb.knnMatch(des1_orb, des2_orb, k=2)

good_orb = []

for m, n in matches_orb:
    if m.distance < 0.75 * n.distance:
        good_orb.append(m)

img_orb = cv2.drawMatches(
    imagen1, kp1_orb,
    imagen2, kp2_orb,
    good_orb, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

In [7]:
def mostrar_imagen(img, nombre, ancho=1200):
    h, w = img.shape[:2]
    escala = ancho / w
    nueva_dim = (ancho, int(h * escala))
    img_resized = cv2.resize(img, nueva_dim)
    cv2.imshow(nombre, img_resized)


In [16]:
mostrar_imagen(img_sift, "Matches con SIFT")
mostrar_imagen(img_orb, "Matches con ORB")

cv2.waitKey(0)
cv2.destroyAllWindows()